In [10]:
%matplotlib notebook

import os
import numpy as np
import matplotlib.pyplot as plt

# plt.rcParams['pgf.texsystem'] = 'pdflatex'
# plt.rcParams.update({'font.family': 'serif', 'font.size': 18,
#                      'figure.titlesize' : 28,
#                      'axes.labelsize': 20,'axes.titlesize': 24,
#                      'legend.fontsize': 20, 'legend.handlelength': 2})
# plt.rcParams['text.usetex'] = True
plt.rcParams["figure.figsize"] = (12, 8)

from matplotlib.colors import Normalize # colormaps
from scipy.optimize import curve_fit # curve fitting
from scipy.signal.windows import get_window # FFT windowing

from tools import datareading, rivuletfinding, utility, datasaving

In [11]:
# Dataset selection
dataset = 'f50_d95_110'
dataset_path = os.path.join('../', dataset)
print('Available acquisitions:', datareading.find_available_videos(dataset_path))

Available acquisitions: ['f50d100_0', 'f50d100_1000', 'f50d100_1500', 'f50d100_2000', 'f50d100_2400', 'f50d100_3000', 'f50d100_3500', 'f50d105_0', 'f50d105_1000', 'f50d105_1500', 'f50d105_2000', 'f50d105_2400', 'f50d105_3000', 'f50d105_3500', 'f50d110_0', 'f50d110_1000', 'f50d110_1500', 'f50d110_2000', 'f50d110_2400', 'f50d110_3000', 'f50d90_0', 'f50d90_1000', 'f50d90_1500', 'f50d90_2000', 'f50d90_2400', 'f50d90_3000', 'f50d90_3500', 'f50d95_0', 'f50d95_1000', 'f50d95_1500', 'f50d95_2000', 'f50d95_2400', 'f50d95_3000', 'f50d95_3500']


In [15]:
# Parameters definition
framenumbers = None
roi = None, None, None, None  #start_x, start_y, end_x, end_y

xprobe = None
xprobeinterval = None
fft_window = None
excitation_frequency = None
deltaf = None

if dataset == 'f50_d95_110':
    xprobe = None
    xprobeinterval = [1200, 1250]
    fft_window = 'hann'
    excitation_frequency = 50
    deltaf = 10

In [16]:
acquisitions = datareading.find_available_videos(dataset_path)

import re

acquisitions.sort(key=lambda acqu: re.search('d(.+?)_', acqu).group(1).zfill(3) + re.search('_(.+)', acqu).group(1))

print(acquisitions)

['f50d90_0', 'f50d90_1000', 'f50d90_1500', 'f50d90_2000', 'f50d90_2400', 'f50d90_3000', 'f50d90_3500', 'f50d95_0', 'f50d95_1000', 'f50d95_1500', 'f50d95_2000', 'f50d95_2400', 'f50d95_3000', 'f50d95_3500', 'f50d100_0', 'f50d100_1000', 'f50d100_1500', 'f50d100_2000', 'f50d100_2400', 'f50d100_3000', 'f50d100_3500', 'f50d105_0', 'f50d105_1000', 'f50d105_1500', 'f50d105_2000', 'f50d105_2400', 'f50d105_3000', 'f50d105_3500', 'f50d110_0', 'f50d110_1000', 'f50d110_1500', 'f50d110_2000', 'f50d110_2400', 'f50d110_3000']


In [17]:
zeta0s = np.empty(len(acquisitions), dtype=float)
zeta0_us = np.empty(len(acquisitions), dtype=float)

for i_acq, acquisition in enumerate(acquisitions):
    print(acquisition)
    acquisition_path = os.path.join(dataset_path, acquisition)
    datareading.is_this_a_video(acquisition_path)

    # Data fetching
    frames = datareading.get_frames(acquisition_path, framenumbers = framenumbers, subregion=roi)
    length, height, width = frames.shape
    t = datareading.get_times(acquisition_path, framenumbers=framenumbers, unit='s')
    acquisition_frequency = datareading.get_acquisition_frequency(acquisition_path, unit="Hz")

    # Purification
    rivs = datasaving.fetch_or_generate_data('cos', dataset, acquisition, framenumbers=framenumbers, roi=roi)
    riv_x = np.linspace(0, width, rivs.shape[1], endpoint=False)

    # get temporal mean
    if xprobe is not None:
        latdis = rivs[:, np.argmin(np.abs(riv_x - xprobe))]
    else:
        ix1 = np.argmin(np.abs(riv_x - xprobeinterval[0]))
        ix2 = np.argmin(np.abs(riv_x - xprobeinterval[1]))
        latdis = rivs[:, ix1:ix2].mean(axis=1)

    # fft
    f = np.fft.rfftfreq(len(t), 1/acquisition_frequency)
    df = np.abs(f[1] - f[0])
    fmin, fmax = f.min() - df/2, f.max() + df/2

    latdis_treated = latdis.copy() # the windowed, FFT ready data
    latdis_treated -= np.mean(latdis)* (1-1e-8) # offset removal
    latdis_treated *= get_window(fft_window, len(t)) # windowing

    latdis_hat = np.fft.rfft(latdis_treated, norm='backward')
    latdis_pw = np.abs(latdis_hat)**2

    crit = (f > excitation_frequency-deltaf) * (f < excitation_frequency+deltaf)

    energy_total = latdis_pw.sum()
    energy_around_excitation = latdis_pw[crit].sum()
    print(f'Proportion of energy at the excitation: {round(100 * energy_around_excitation / energy_total, 1)}%')

    spectral_energy = energy_around_excitation
    field_factor = len(t)/2 # normalization factor which depends on the FFT convention used. Here we are 'backward', numpy's default convention
    window_factor = 1/np.sqrt((get_window(fft_window, len(t))**2).sum()/len(t)) # normalization factor which depends on the window used. For example: 1 for boxcar, (8/3)^(d/2) for hann
    fft_factor = field_factor / window_factor

    zeta0 = np.sqrt(spectral_energy) / fft_factor
    print(f'amplitude: {round(zeta0,2)} px')

    zeta0s[i_acq] = zeta0

    print(f'\tzeta0 measured for {acquisition}: {round(zeta0,3)}')

f50d90_0
Proportion of energy at the excitation: 7.5%
amplitude: 0.04 px
	zeta0 measured for f50d90_0: 0.044
f50d90_1000
Proportion of energy at the excitation: 99.4%
amplitude: 2.28 px
	zeta0 measured for f50d90_1000: 2.276
f50d90_1500
Proportion of energy at the excitation: 99.8%
amplitude: 3.28 px
	zeta0 measured for f50d90_1500: 3.281
f50d90_2000
Proportion of energy at the excitation: 99.9%
amplitude: 4.21 px
	zeta0 measured for f50d90_2000: 4.213
f50d90_2400
Proportion of energy at the excitation: 99.9%
amplitude: 4.98 px
	zeta0 measured for f50d90_2400: 4.981
f50d90_3000
Proportion of energy at the excitation: 99.4%
amplitude: 5.01 px
	zeta0 measured for f50d90_3000: 5.012
f50d90_3500
Proportion of energy at the excitation: 97.4%
amplitude: 5.2 px
	zeta0 measured for f50d90_3500: 5.203
f50d95_0
Proportion of energy at the excitation: 14.1%
amplitude: 0.04 px
	zeta0 measured for f50d95_0: 0.043
f50d95_1000
Proportion of energy at the excitation: 99.2%
amplitude: 2.29 px
	zeta0 me

In [18]:
for k in range(len(zeta0s)):
    print(f'{acquisitions[k]}\t{zeta0s[k]}')

f50d90_0	0.044332214702398016
f50d90_1000	2.2757314945520104
f50d90_1500	3.2813059982921264
f50d90_2000	4.212935942415097
f50d90_2400	4.980894609737116
f50d90_3000	5.011836374042271
f50d90_3500	5.202854692598019
f50d95_0	0.04318972120946016
f50d95_1000	2.288331279099869
f50d95_1500	3.2981915710127474
f50d95_2000	4.17962492048033
f50d95_2400	4.924469333265303
f50d95_3000	5.942628865421885
f50d95_3500	6.000122574686619
f50d100_0	0.04026029630843862
f50d100_1000	2.1901705559332743
f50d100_1500	3.116172218744683
f50d100_2000	3.9719793816120403
f50d100_2400	4.6663379127637565
f50d100_3000	5.734721732290014
f50d100_3500	5.940987984136671
f50d105_0	0.0406758858417534
f50d105_1000	1.9111527103976977
f50d105_1500	2.6021015018148943
f50d105_2000	3.248466459317559
f50d105_2400	4.025615958541723
f50d105_3000	4.521175037528573
f50d105_3500	5.208898604278164
f50d110_0	0.026696101664537304
f50d110_1000	1.83567549811345
f50d110_1500	2.696737399366101
f50d110_2000	3.219445436698042
f50d110_2400	3.95793